# LlamaIndex Context Snapshot Extraction

This notebook demonstrates how to extract a context snapshot from a single LlamaIndex workflow run.

## Setup and Imports


In [13]:
import asyncio
import json
from datetime import datetime
from typing import List, Dict, Any

from llama_index.core.workflow import Workflow, StartEvent, StopEvent, step, Event
from llama_index.core.workflow.checkpointer import WorkflowCheckpointer
from llama_index.core.workflow.context import Context

print("✅ All imports successful!")
print(f"📅 Current time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


✅ All imports successful!
📅 Current time: 2025-09-12 09:15:35


## Simple Chat Workflow with Context Snapshot Extraction

This section shows how to extract a context snapshot from a single workflow run.


In [14]:
# Define events for simple chat demo
class UserMessageEvent(Event):
    user_message: str
    session_id: str
    message_count: int = 0

class BotResponseEvent(Event):
    user_message: str
    bot_response: str
    session_id: str
    conversation_history: List[Dict] = []

print("📋 Event classes defined for simple chat demo")


📋 Event classes defined for simple chat demo


## Simple Chat Workflow

This section shows a simple chat workflow that stores conversation history in context.


In [15]:
# Create simple chat workflow using class-based approach
class SimpleChatWorkflow(Workflow):
    def __init__(self):
        super().__init__(timeout=60, verbose=True)
    
    @step
    async def process_user_message(self, ev: StartEvent, ctx: Context) -> UserMessageEvent:
        """Step 1: Process user message and load conversation history"""
        print("\n🔄 STEP 1: Processing user message...")
        print(f"📥 Input data: {ev.input_data}")
        
        # Load conversation history from context
        conversation_history = await ctx.get("conversation_history", [])
        print(f"📚 Loaded {len(conversation_history)} previous messages from context")
        
        user_message = ev.input_data.get("user_message", "Hello!")
        session_id = ev.input_data.get("session_id", "demo_session")
        
        result = UserMessageEvent(
            user_message=user_message,
            session_id=session_id,
            message_count=len(conversation_history) + 1
        )
        
        print(f"✅ User message processed")
        print(f"📤 Output: {result}")
        
        return result
    
    @step
    async def generate_bot_response(self, ev: UserMessageEvent, ctx: Context) -> StopEvent:
        """Step 2: Generate bot response and update conversation history"""
        print("\n🔄 STEP 2: Generating bot response...")
        print(f"📥 Input: {ev.user_message} (message #{ev.message_count})")
        
        # Get conversation history from context
        conversation_history = await ctx.get("conversation_history", [])
        print(f"📚 Processing with {len(conversation_history)} previous messages in context")
        
        # Generate context-aware response
        if ev.message_count == 1:
            bot_response = "Hello! This is our first conversation! How can I help you today?"
        elif ev.message_count == 2:
            bot_response = "Nice to see you again! What would you like to know about?"
        else:
            bot_response = f"Thanks for message #{ev.message_count}! I'm here to help with any questions you have."
        
        # Add new message to conversation history
        new_message = {
            "user_message": ev.user_message,
            "bot_response": bot_response,
            "timestamp": datetime.now().isoformat(),
            "message_number": ev.message_count
        }
        
        # Update conversation history in context
        updated_history = conversation_history + [new_message]
        await ctx.set("conversation_history", updated_history)
        
        print(f"💾 Updated context with {len(updated_history)} total messages")
        print(f"🤖 Bot response: {bot_response}")
        
        result = StopEvent(result={
            "session_id": ev.session_id,
            "user_message": ev.user_message,
            "bot_response": bot_response,
            "conversation_history": updated_history,
            "message_count": ev.message_count,
            "completed_at": datetime.now().isoformat()
        })
        
        print(f"✅ Bot response generated")
        print(f"📤 Output: {result.result}")
        
        return result

# Create workflow instance
chat_workflow = SimpleChatWorkflow()
print("🔧 Simple chat workflow created with 2 steps")


🔧 Simple chat workflow created with 2 steps


In [16]:
# Demonstrate context snapshot extraction from a single workflow run
async def extract_context_snapshot():
    global checkpointer  # Make checkpointer global so it can be accessed in other cells
    print("\n" + "="*60)
    print("🚀 CONTEXT SNAPSHOT EXTRACTION")
    print("="*60)
    
    # Create checkpointer
    checkpointer = WorkflowCheckpointer(workflow=chat_workflow)
    
    print("\n📋 Running a single workflow instance...")
    print("🎯 We will extract the context snapshot after the workflow completes")
    print(f"🔧 Checkpointer created for workflow: {type(chat_workflow).__name__}")
    print(f"📊 Initial checkpoints: {len(checkpointer.checkpoints)}")
    
    # Run workflow with a single message
    handler = checkpointer.run(input_data={
        "user_message": "Hello! Can you help me with Python?",
        "session_id": "demo_session_123"
    })
    
    print("\n⏳ Waiting for workflow to complete...")
    result = await handler
    
    print("\n🎉 Workflow completed!")
    print(f"📊 Workflow result: {result}")
    
    # Check if checkpoints were created
    print("\n📊 Checking checkpoints...")
    print(f"📋 Total checkpoints: {len(checkpointer.checkpoints)}")
    for session_id, checkpoints in checkpointer.checkpoints.items():
        print(f"  Session: {session_id}")
        print(f"  Number of checkpoints: {len(checkpoints)}")
        for i, checkpoint in enumerate(checkpoints):
            print(f"    Checkpoint {i+1}: {checkpoint.last_completed_step}")
    
    # Extract context snapshot
    print("\n💾 Extracting context snapshot...")
    context_snapshot = handler.ctx.to_dict()
    
    print(f"✅ Context snapshot extracted with {len(context_snapshot)} keys")
    print(f"🔑 Top-level keys: {list(context_snapshot.keys())}")
    
    # Show conversation history from context
    conversation_history = []
    if "state" in context_snapshot and "state_data" in context_snapshot["state"]:
        state_data = context_snapshot["state"]["state_data"]
        if "_data" in state_data and "conversation_history" in state_data["_data"]:
            # The conversation_history might be stored as a JSON string, so we need to parse it
            raw_history = state_data["_data"]["conversation_history"]
            if isinstance(raw_history, str):
                try:
                    conversation_history = json.loads(raw_history)
                except json.JSONDecodeError:
                    conversation_history = []
            else:
                conversation_history = raw_history
    
    print(f"\n📚 Conversation history from context ({len(conversation_history)} messages):")
    if conversation_history:
        for i, msg in enumerate(conversation_history, 1):
            print(f"  {i}. User: {msg['user_message']}")
            print(f"     Bot: {msg['bot_response']}")
            print(f"     Time: {msg['timestamp']}")
            print()
    else:
        print("  No conversation history found in context")
    
    # Show context structure
    print("\n🔍 Context Snapshot Structure:")
    print(f"  State type: {context_snapshot.get('state', {}).get('state_type', 'Unknown')}")
    print(f"  Queues: {list(context_snapshot.get('queues', {}).keys())}")
    print(f"  Accepted events: {context_snapshot.get('accepted_events', [])}")
    print(f"  Broker log entries: {len(context_snapshot.get('broker_log', []))}")
    
    return context_snapshot

# Run the demonstration
context_snapshot = await extract_context_snapshot()



🚀 CONTEXT SNAPSHOT EXTRACTION

📋 Running a single workflow instance...
🎯 We will extract the context snapshot after the workflow completes
🔧 Checkpointer created for workflow: SimpleChatWorkflow
📊 Initial checkpoints: 0

⏳ Waiting for workflow to complete...
Running step process_user_message

🔄 STEP 1: Processing user message...
📥 Input data: {'user_message': 'Hello! Can you help me with Python?', 'session_id': 'demo_session_123'}
📚 Loaded 0 previous messages from context
✅ User message processed
📤 Output: user_message='Hello! Can you help me with Python?' session_id='demo_session_123' message_count=1
Step process_user_message produced event UserMessageEvent
Running step generate_bot_response

🔄 STEP 2: Generating bot response...
📥 Input: Hello! Can you help me with Python? (message #1)
📚 Processing with 0 previous messages in context
💾 Updated context with 1 total messages
🤖 Bot response: Hello! This is our first conversation! How can I help you today?
✅ Bot response generated
📤 Outp

C:\Users\vince.szabo_hiflylab\AppData\Local\Temp\ipykernel_23468\4148767160.py:9: DeprecationWarning: WorkflowCheckpointer is deprecated and will be removed in a future version.
  checkpointer = WorkflowCheckpointer(workflow=chat_workflow)
C:\Users\vince.szabo_hiflylab\AppData\Local\Temp\ipykernel_23468\2996156014.py:13: DeprecationWarning: Context.get() is deprecated. Use 'await ctx.store.get()' instead.
  conversation_history = await ctx.get("conversation_history", [])
C:\Users\vince.szabo_hiflylab\AppData\Local\Temp\ipykernel_23468\2996156014.py:37: DeprecationWarning: Context.get() is deprecated. Use 'await ctx.store.get()' instead.
  conversation_history = await ctx.get("conversation_history", [])
C:\Users\vince.szabo_hiflylab\AppData\Local\Temp\ipykernel_23468\2996156014.py:58: DeprecationWarning: Context.set(key, value) is deprecated. Use 'await ctx.store.set(key, value)' instead.
  await ctx.set("conversation_history", updated_history)


In [17]:
# Resume workflow from the second checkpoint
async def resume_from_checkpoint():
    print("\n" + "="*60)
    print("🔄 RESUMING FROM CHECKPOINT")
    print("="*60)
    
    # Use the same checkpointer from the previous run
    print("\n📋 Resuming workflow from the second checkpoint...")
    print("🎯 We will resume from the 'generate_bot_response' step")
    
    # Get the checkpoints from the previous run
    if not checkpointer.checkpoints:
        print("❌ No checkpoints found! Run the first cell first.")
        return
    
    # Get the session ID and checkpoints
    session_id = list(checkpointer.checkpoints.keys())[0]
    checkpoints = checkpointer.checkpoints[session_id]
    
    print(f"📊 Found {len(checkpoints)} checkpoints for session: {session_id}")
    
    # Show available checkpoints
    for i, checkpoint in enumerate(checkpoints):
        print(f"  Checkpoint {i+1}: {checkpoint.last_completed_step}")
    
    # Resume from the second checkpoint (index 1)
    if len(checkpoints) >= 2:
        second_checkpoint = checkpoints[1]  # Second checkpoint
        print(f"\n🔄 Resuming from checkpoint: {second_checkpoint}")
        
        # Resume the workflow from the checkpoint
        resumed_handler = checkpointer.run_from(checkpoint=second_checkpoint)
        
        print("\n⏳ Waiting for resumed workflow to complete...")
        result = await resumed_handler
        
        print("\n🎉 Resumed workflow completed!")
        print(f"📊 Resumed workflow result: {result}")
        
        # Show final checkpoints after resume
        print(f"\n📊 Final checkpoints after resume: {len(checkpointer.checkpoints[session_id])}")
        
    else:
        print("❌ Not enough checkpoints to resume from the second one")

# Run the resume demonstration
await resume_from_checkpoint()



🔄 RESUMING FROM CHECKPOINT

📋 Resuming workflow from the second checkpoint...
🎯 We will resume from the 'generate_bot_response' step
📊 Found 2 checkpoints for session: 4f1cf0cc-a470-4e1f-9bfb-9eeeebcb3dbc
  Checkpoint 1: process_user_message
  Checkpoint 2: generate_bot_response

🔄 Resuming from checkpoint: id_='2cd74e64-6501-4e47-b099-b6609f2ed382' last_completed_step='generate_bot_response' input_event=UserMessageEvent(user_message='Hello! Can you help me with Python?', session_id='demo_session_123', message_count=1) output_event=StopEvent() ctx_state={'state': {'state_data': {'_data': {'conversation_history': '[{"user_message": "Hello! Can you help me with Python?", "bot_response": "Hello! This is our first conversation! How can I help you today?", "timestamp": "2025-09-12T09:15:35.968319", "message_number": 1}]'}}, 'state_type': 'DictState', 'state_module': 'workflows.context.state_store'}, 'streaming_queue': '[]', 'queues': {'_done': '[]', 'generate_bot_response': '[]', 'process_

In [18]:
# Demonstrate modifying checkpoint event data
class NumberEvent(Event):
    number: int
    step_name: str

class ResultEvent(Event):
    original_number: int
    added_number: int
    final_result: int
    step_name: str

class NumberWorkflow(Workflow):
    def __init__(self):
        super().__init__(timeout=60, verbose=True)
    
    @step
    async def first_step(self, ev: StartEvent, ctx: Context) -> NumberEvent:
        """Step 1: Get initial number from input"""
        print("\n🔄 STEP 1: Getting initial number...")
        initial_number = ev.input_data.get("number", 5)
        print(f"📥 Initial number: {initial_number}")
        
        result = NumberEvent(
            number=initial_number,
            step_name="first_step"
        )
        print(f"✅ First step completed with number: {result.number}")
        return result
    
    @step
    async def second_step(self, ev: NumberEvent, ctx: Context) -> StopEvent:
        """Step 2: Add 10 to the number"""
        print("\n🔄 STEP 2: Adding 10 to the number...")
        print(f"📥 Input number: {ev.number}")
        
        added_number = 10
        final_result = ev.number + added_number
        
        print(f"➕ Adding {added_number} to {ev.number}")
        print(f"🎯 Final result: {final_result}")
        
        return StopEvent(result={
            "original_number": ev.number,
            "added_number": added_number,
            "final_result": final_result,
            "step_name": "second_step"
        })

# Create the number workflow
number_workflow = NumberWorkflow()
print("🔧 Number workflow created with 2 steps")

# Demonstrate checkpoint modification
async def demonstrate_checkpoint_modification():
    print("\n" + "="*60)
    print("🔧 CHECKPOINT EVENT MODIFICATION DEMO")
    print("="*60)
    
    # Create checkpointer
    checkpointer = WorkflowCheckpointer(workflow=number_workflow)
    
    print("\n📋 Running workflow with initial number 5...")
    handler = checkpointer.run(input_data={"number": 5})
    result = await handler
    
    print(f"\n🎉 First run completed!")
    print(f"📊 Result: {result}")
    
    # Get checkpoints
    session_id = list(checkpointer.checkpoints.keys())[0]
    checkpoints = checkpointer.checkpoints[session_id]
    
    print(f"\n📊 Found {len(checkpoints)} checkpoints")
    for i, checkpoint in enumerate(checkpoints):
        print(f"  Checkpoint {i+1}: {checkpoint.last_completed_step}")
    
    # Show original checkpoint data
    if len(checkpoints) >= 2:
        first_checkpoint = checkpoints[0]  # First checkpoint (first_step)
        second_checkpoint = checkpoints[1]  # Second checkpoint (second_step)
        print(f"  Checkpoint 1: {first_checkpoint}")
        print(f"\n🔍 Original checkpoint data:")
        print(f"  Checkpoint 1 - Last completed step: {first_checkpoint.last_completed_step}")
        print(f"  Checkpoint 1 - Output event: {first_checkpoint.output_event}")
        print(f"  Checkpoint 2 - Last completed step: {second_checkpoint.last_completed_step}")
        print(f"  Checkpoint 2 - Input event: {second_checkpoint.input_event}")
        
        # MODIFY THE CORRECT CHECKPOINT EVENT DATA
        print(f"\n✏️ MODIFYING CHECKPOINT EVENT DATA...")
        print(f"  Original number in first checkpoint output: {first_checkpoint.output_event.number}")
        
        # Change the number from 5 to 20 in the FIRST checkpoint's output event
        # This is the NumberEvent that gets passed to the second step
        first_checkpoint.output_event.number = 20
        print(f"  Modified number in first checkpoint output: {first_checkpoint.output_event.number}")
        
        # Resume from the FIRST checkpoint (so it re-runs the second step with modified data)
        print(f"\n🔄 Resuming from first checkpoint...")
        print(f"  Resuming from step: {first_checkpoint.last_completed_step}")
        print(f"  Using modified number: {first_checkpoint.output_event.number}")
        print(f"  This will re-run the second step with the modified number!")
        
        resumed_handler = checkpointer.run_from(checkpoint=first_checkpoint)
        modified_result = await resumed_handler
        
        print(f"\n🎉 Resumed workflow completed!")
        print(f"📊 Modified result: {modified_result}")
        
        # Compare results
        print(f"\n📊 COMPARISON:")
        print(f"  Original run result: {result}")
        print(f"  Modified run result: {modified_result}")
        print(f"  ✅ The result changed because we modified the input event data!")
        
    else:
        print("❌ Not enough checkpoints to demonstrate modification")

# Run the demonstration
await demonstrate_checkpoint_modification()


🔧 Number workflow created with 2 steps

🔧 CHECKPOINT EVENT MODIFICATION DEMO

📋 Running workflow with initial number 5...
Running step first_step

🔄 STEP 1: Getting initial number...
📥 Initial number: 5
✅ First step completed with number: 5
Step first_step produced event NumberEvent
Running step second_step

🔄 STEP 2: Adding 10 to the number...
📥 Input number: 5
➕ Adding 10 to 5
🎯 Final result: 15
Step second_step produced event StopEvent

🎉 First run completed!
📊 Result: {'original_number': 5, 'added_number': 10, 'final_result': 15, 'step_name': 'second_step'}

📊 Found 2 checkpoints
  Checkpoint 1: first_step
  Checkpoint 2: second_step
  Checkpoint 1: id_='15e0ff9c-4a28-48af-a76f-7c314a9d8b55' last_completed_step='first_step' input_event=StartEvent() output_event=NumberEvent(number=5, step_name='first_step') ctx_state={'state': {'state_data': {'_data': {}}, 'state_type': 'DictState', 'state_module': 'workflows.context.state_store'}, 'streaming_queue': '[]', 'queues': {'_done': '[]', 

C:\Users\vince.szabo_hiflylab\AppData\Local\Temp\ipykernel_23468\733539094.py:60: DeprecationWarning: WorkflowCheckpointer is deprecated and will be removed in a future version.
  checkpointer = WorkflowCheckpointer(workflow=number_workflow)
